# RespiCoV sequencing analysis by known sequence

Analyze fastq file(s) from nanopore sequencing output using known sequences to match against.

Run here on my first two RespiCoV sequencing attempts where I knew (from gel) that I had a lot of mis-priming and low read counts. Both attempts used 50-54 different samples divided into 8 different barcoded pools. The RC2 run

**Goals:**
 * Precisely identify target matches per input sample given known sequences
 * Run relatively quickly on a single machine and scale linearly with input sequence
 * Enable iterative exploration of the data (cache the most expensive operations)
 * Support multiple targets per sample (pooled samples)
 * Work with either ligration or tagmentation chemistry
 * Don't rely on precise primer-pair identification

**TODO:**
 * Look into unexpected matches - just barcode failure or possibly a different virus all together?
 
**Non-goals / future work elsewhere:**
 * Analyze mis-priming or PCR efficiency (see RCMatchPrimers).
 * Support more matching read than can fit into RAM at once

## Initialization and configuration

In [1]:
import random
from Bio.Seq import Seq
from Bio import SeqIO
import matplotlib_inline.backend_inline
import os
import pandas as pd
import RCUtils

%load_ext autoreload
%autoreload 1
%aimport RCUtils

# Get high-dpi output for retina displays
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

fastQBaseDirs = {
    "RC1": "../RespiCov-1/20221204_2344_MN41817_FAV39017_1bf37150/fastq_pass/",
    "RC2": "../RespiCov-2/20230430_1642_MN41817_APC888_8b249272/fastq_pass/"}

seqDir = "myseqs"

pd.options.display.max_rows = 50
pd.options.display.min_rows = 25

# Read in all the reference sequences
seqs = []
for file in sorted(filter(lambda f: f.endswith(".fastq"), os.listdir(seqDir))):  
    for seq in SeqIO.parse(os.path.join(seqDir, file), "fastq"):
        seqs.append(seq)
print("Read %i sequences" % (len(seqs)))


Read 3 sequences


## Process all the reads

In [2]:
import time

def getSeqMatches():
    for (sample, fastQDir) in RCUtils.getAllSampleDirs(fastQBaseDirs):
        print("Processing " + sample + ": ", end="")
        reads = 0
        start = time.process_time()
        matchingReads = []

        for read in RCUtils.getReads(fastQDir):
            reads += 1
            matches = RCUtils.seqMatch(read, seqs)
            if len(matches) > 0:
                matchingReads.append((read, matches))

        elapsed = time.process_time() - start
        print("  %i / %i reads, %.2fs" % (len(matchingReads), reads, elapsed))
        yield (sample, matchingReads)

# Store all the results in a list
seqMatches = list(getSeqMatches())
print("Processed %i samples" % len(seqMatches))

Processing RC1-barcode01:   0 / 647 reads, 2.07s
Processing RC1-barcode02:   1 / 692 reads, 2.13s
Processing RC1-barcode03:   16 / 436 reads, 1.67s
Processing RC1-barcode04:   0 / 624 reads, 1.95s
Processing RC1-barcode05:   0 / 682 reads, 2.14s
Processing RC1-barcode06:   26 / 771 reads, 2.69s
Processing RC1-barcode07:   17 / 809 reads, 2.71s
Processing RC1-barcode08:   0 / 894 reads, 2.50s
Processing RC2-barcode01:   0 / 502 reads, 2.20s
Processing RC2-barcode02:   0 / 240 reads, 1.00s
Processing RC2-barcode03:   0 / 369 reads, 1.53s
Processing RC2-barcode04:   35 / 251 reads, 1.49s
Processing RC2-barcode05:   0 / 179 reads, 0.73s
Processing RC2-barcode06:   0 / 341 reads, 1.44s
Processing RC2-barcode07:   4 / 408 reads, 1.73s
Processing RC2-barcode08:   71 / 299 reads, 2.10s
Processed 16 samples


## Dump the match details per target

In [4]:
from collections import defaultdict
import statistics

seqMatchesPerTarget = {}
for (sample, matchingReads) in seqMatches:
    perTarget = defaultdict(list)
    for (read, matches) in matchingReads:
        mr = matches[0].score / len(matches[0].target)
        mrdiff = mr
        if (len(matches) > 1):
            mrdiff = mr - (matches[1].score / len(matches[1].target))
        matchLen = matches[0].targetEnd - matches[0].targetStart
        perTarget[matches[0].target.id].append({"mr":mr, "mrdiff":mrdiff, "len": matchLen, "read":read, "match":matches[0]})
    if (len(perTarget) > 0):
        seqMatchesPerTarget[sample] = perTarget

for (sample, perTarget) in seqMatchesPerTarget.items():
    print(sample)
    for (target, matches) in perTarget.items():
        mrs = list(map(lambda m: m["mr"], matches))
        print("  %s: %i matches, avg mr=%.0f%%" % (target, len(matches), statistics.median(mrs) * 100))

RC1-barcode02
  S28-RVA-23: 1 matches, avg mr=42%
RC1-barcode03
  S28-RVA-23: 16 matches, avg mr=81%
RC1-barcode06
  S44-RVA-56: 26 matches, avg mr=79%
RC1-barcode07
  S48-RVC-1: 16 matches, avg mr=79%
  S44-RVA-56: 1 matches, avg mr=82%
RC2-barcode04
  S48-RVC-1: 34 matches, avg mr=85%
  S28-RVA-23: 1 matches, avg mr=85%
RC2-barcode07
  S28-RVA-23: 4 matches, avg mr=79%
RC2-barcode08
  S44-RVA-56: 12 matches, avg mr=79%
  S28-RVA-23: 59 matches, avg mr=83%


## What's with the RC1-barcode02 result?
Is it just a barcode failure, or might it actually be a different Rhinovirus?

In [11]:
def printSpecificReadSeq(sample, target):
    matchInfo = max(seqMatchesPerTarget[sample][target], key=lambda m: m["mr"])
    print(">%s %s %.0f%%" % (sample, target, matchInfo["mr"] * 100))
    print(matchInfo["read"][matchInfo["match"].readStart:matchInfo["match"].readEnd].seq)

printSpecificReadSeq("RC1-barcode02", "S28-RVA-23")
printSpecificReadSeq("RC1-barcode03", "S28-RVA-23")

>RC1-barcode02 S28-RVA-23 42%
AACACGGACACCAAAGTAGTTGGTCCCATCCCGCAATTGCTCGTTACGACTAGAAACACTGGATTGTGCGCTCTAGCTGCAGGGTTAAGGTTAGCCACATTCGGGGGCCGGGAGGACTCAAAGCGACACACGGGGCTCTTCACACCCTATCCATTGTATGGCTTCACACCCAGAGGGT
>RC1-barcode03 S28-RVA-23 91%
AACACGGACACCCAAAGTAGTTGGTCCCATCCCGCAATTGCTCGTTACGACTAGAGAAACACACTGGATTGTGCGCTCTAGCTGCAGGGATTTGAGTTAGCCACATTCCAGGGGCCGGAGGACTCAAAGCGAGCACACGGGGCTCTTCACACCCTGTCCATTGTATGGCTTCACACCCAGAGGGTGTGCAGGCAGCCACGCAGGCTAGAACACTGTCGCCAGTGGGGAGTTTCTAGCCTCTGGCTCTGCCAGGTCTACCAGGGGGAATGGTGGAGCGACCAGCCAGACAACTTCCAGAGTACTACTAAGCTTTGCGTAGGCGCTTTGCGGATAACGATCGCAGCTGTTTTTGCCCTGGTGGGGCATATCAACTTTGACCGGGGAAACAGGAG


Doing a blast for these sequences we can see that it's indeed both RV-A23, but the first is truncated, only the last half.